In [1]:
import pandas as pd
import numpy as np

# Ammonia data clean-up

In [2]:
ammonia_df = pd.read_excel('./statistic_id1266392_ammonia-plant-production-capacity-in-the-us-2022.xlsx', sheet_name='Data', header=None, skiprows=5, usecols=[1,2])
ammonia_df.rename(columns={1:'Plant',2:'Capacity'}, inplace=True)
ammonia_df['City'] = ammonia_df.apply(lambda x:x['Plant'].split('(')[1].split(',')[0], axis=1)
ammonia_df['State'] = ammonia_df.apply(lambda x:x['Plant'].split('(')[1][:-1].split(',')[1], axis=1)
ammonia_df['Plant'] = ammonia_df.apply(lambda x:x['Plant'].split('(')[0][:-1], axis=1)
ammonia_df['Capacity (tNH3/year)'] = ammonia_df['Capacity']*1e3 # Original data in thousands MT
ammonia_df.drop(columns=['Capacity'], inplace=True)
# Create id with plant name and city
ammonia_df['plant_id'] = ammonia_df.apply(lambda x: x['Plant'][:2]+'_'+x['City'][:2], axis=1)
ammonia_df

,Plant,City,State,Capacity (tNH3/year),plant_id
0,CF Industries Inc.,Donaldsonville,LA,3935000.0,CF_Do
1,CF Industries Inc.,Port Neal,IA,1150000.0,CF_Po
2,CF Industries Inc.,Verdigris,OK,1100000.0,CF_Ve
3,Koch Industries Inc.,Enid,OK,1015000.0,Ko_En
4,Iowa Fertilizer Co. OCI N.V.,Wever,IA,915000.0,Io_We
5,Dyno Nobel Inc.,Waggaman,LA,800000.0,Dy_Wa
6,Nutrien,Augusta,GA,765000.0,Nu_Au
7,Nutrien,Lima,OH,725000.0,Nu_Li
8,Yara/BASF JV,Freeport,TX,725000.0,Ya_Fr
9,Nutrien,Geismar,LA,535000.0,Nu_Ge


# Hydrogen equivalent demand

In [3]:
# Values in modeling paper based on LHV of materials
h2_lhv = pd.read_excel('./fuels_HHV_industry_heat.xlsx', sheet_name='lhv', skiprows= 1, index_col=0).loc['Hydrogen', 'HHV (MJ/kg)']
h2_to_nh3 = 23.67*1e3 #MJ/tNH3
h2_to_nh3_ratio = h2_to_nh3/h2_lhv #kgH2/tNH3
print('H2 to NH3 ratio: {} kgH2/tNH3'.format(h2_to_nh3_ratio))

ammonia_df['H2 Dem. (kg/year)'] = ammonia_df['Capacity (tNH3/year)']*h2_to_nh3_ratio
elec_to_nh3 = 0.119 # MWh/tNH3
ammonia_df['Electricity demand (MWe)'] = elec_to_nh3*ammonia_df['Capacity (tNH3/year)']/(365*24)


H2 to NH3 ratio: 504.712354471406 kgH2/tNH3


## Save cleaned-up demand

In [4]:
ammonia_df.to_excel('../h2_demand_ammonia_us_2022.xlsx', sheet_name='processed', index=False)